In [12]:
import pandas as pd 
import statsmodels.api as sm

# Función auxiliar para "escapar" nombres de variables no válidos en fórmulas
def safe_var(var):
    """Si el nombre de la variable no es un identificador válido, lo envuelve con backticks (`...`)."""
    if var.isidentifier():
        return var
    else:
        return "`{}`".format(var)

# Cargar el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')
df = df.drop(columns=['gidenpac', 'fllegada','ds_estancia', 'ds_post_oper', 'ds_vivo_alta' ])

# Limpiar los nombres de las columnas eliminando espacios en blanco (incluidos los intermedios)
df.columns = df.columns.str.replace(" ", "")

# Verificar las columnas
print("Columnas del DataFrame:", df.columns.tolist())

# Definir la variable respuesta (target)
response = "ds_pre_oper"

# -------------------------------
# Función para Forward Selection
# -------------------------------
def forward_selected(data, response):
    """
    Realiza la selección hacia adelante (forward selection) usando el criterio AIC.
    
    Parameters:
        data (DataFrame): El conjunto de datos.
        response (str): El nombre de la variable dependiente.
    
    Returns:
        model: El modelo final ajustado.
        selected: Lista de variables seleccionadas.
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    
    while remaining:
        scores_with_candidates = []
        for candidate in remaining:
            # Construir la fórmula usando safe_var para cada variable
            if selected:
                predictors = " + ".join([safe_var(var) for var in selected + [candidate]])
            else:
                predictors = safe_var(candidate)
            formula = "{} ~ {}".format(safe_var(response), predictors)
            try:
                score = sm.OLS.from_formula(formula, data).fit().aic
            except Exception as e:
                # Si falla la fórmula se ignora esta variable y se informa el error
                print("Error en la fórmula '{}': {}".format(formula, e))
                continue
            scores_with_candidates.append((score, candidate))
        if not scores_with_candidates:
            break
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    
    final_formula = "{} ~ {}".format(safe_var(response), " + ".join([safe_var(var) for var in selected]))
    model = sm.OLS.from_formula(final_formula, data).fit()
    return model, selected

# Aplicar forward selection
model_forward, selected_forward = forward_selected(df, response)
print("Variables seleccionadas por Forward Selection:")
print(selected_forward)
print("\nResumen del modelo (Forward Selection):")
print(model_forward.summary())

# -------------------------------
# Función para Backward Elimination
# -------------------------------
'''def backward_elimination(data, response, significance_level=0.05):
    """
    Realiza la eliminación hacia atrás (backward elimination) eliminando variables con p-valor mayor al nivel de significancia.
    
    Parameters:
        data (DataFrame): El conjunto de datos.
        response (str): El nombre de la variable dependiente.
        significance_level (float): Nivel de significancia para eliminar variables.
    
    Returns:
        model: El modelo final ajustado.
        selected: Lista de variables que permanecen en el modelo.
    """
    selected = list(data.columns)
    selected.remove(response)
    
    while True:
        predictors = " + ".join([safe_var(var) for var in selected])
        formula = "{} ~ {}".format(safe_var(response), predictors)
        model = sm.OLS.from_formula(formula, data).fit()
        # Excluir el intercepto al evaluar los p-valores
        pvalues = model.pvalues.iloc[1:]
        max_pvalue = pvalues.max()
        if max_pvalue > significance_level:
            worst_feature = pvalues.idxmax()
            # Buscar la variable original que, al aplicar safe_var, coincide con worst_feature
            for var in selected:
                if safe_var(var) == worst_feature:
                    selected.remove(var)
                    break
        else:
            break
    
    final_formula = "{} ~ {}".format(safe_var(response), " + ".join([safe_var(var) for var in selected]))
    model = sm.OLS.from_formula(final_formula, data).fit()
    return model, selected

# Aplicar backward elimination
model_backward, selected_backward = backward_elimination(df, response)
print("\nVariables seleccionadas por Backward Elimination:")
print(selected_backward)
print("\nResumen del modelo (Backward Elimination):")
print(model_backward.summary())
'''


Columnas del DataFrame: ['itipsexo', 'gsitalta', 'iotrocen', 'gdiagalt', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_pre_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'ds_comorbilidades', 'ds_ulcera_presion']
Variables seleccionadas por Forward Selection:
['ds_dia_semana_llegada', 'ds_ulcera_presion', 'ds_mes_llegada', 'ds_insuficiencia_respiratoria', 'ds_alergia_medicamentosa', 'ds_turno', 'iotrocen']

Resumen del modelo (Forward Selection):
                            OLS Regression Results                            
Dep. Variable:            ds_pre_oper   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   

'def backward_elimination(data, response, significance_level=0.05):\n    """\n    Realiza la eliminación hacia atrás (backward elimination) eliminando variables con p-valor mayor al nivel de significancia.\n    \n    Parameters:\n        data (DataFrame): El conjunto de datos.\n        response (str): El nombre de la variable dependiente.\n        significance_level (float): Nivel de significancia para eliminar variables.\n    \n    Returns:\n        model: El modelo final ajustado.\n        selected: Lista de variables que permanecen en el modelo.\n    """\n    selected = list(data.columns)\n    selected.remove(response)\n    \n    while True:\n        predictors = " + ".join([safe_var(var) for var in selected])\n        formula = "{} ~ {}".format(safe_var(response), predictors)\n        model = sm.OLS.from_formula(formula, data).fit()\n        # Excluir el intercepto al evaluar los p-valores\n        pvalues = model.pvalues.iloc[1:]\n        max_pvalue = pvalues.max()\n        if max

In [13]:
import pandas as pd
import statsmodels.api as sm

# Cargar el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')

# Eliminar columnas no relevantes
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_post_oper', 'ds_vivo_alta'])
df = pd.get_dummies(df, columns=['gdiagalt'], drop_first=True)

# Asegurarse de que la variable respuesta es numérica
df["ds_pre_oper"] = pd.to_numeric(df["ds_pre_oper"], errors='coerce')
df = df.dropna(subset=["ds_pre_oper"])

# Separar variables predictoras (X) y respuesta (y)
X = df.drop(columns=["ds_pre_oper"])
y = df["ds_pre_oper"]

# Convertir las variables categóricas a dummies (las que ya son numéricas se mantienen)
X = pd.get_dummies(X, drop_first=True)

# Forzar que todas las columnas sean numéricas
X = X.apply(pd.to_numeric, errors='coerce')

# Eliminar filas con NaN en X y sincronizar y
X = X.dropna()
y = y.loc[X.index]

# Forzar que X e y sean de tipo float
X = X.astype(float)
y = y.astype(float)

# Agregar la constante para el intercepto
X = sm.add_constant(X)

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor al nivel sl.
    
    Parameters:
        X (DataFrame): Variables predictoras (incluyendo la constante).
        y (Series): Variable respuesta.
        sl (float): Nivel de significancia para retener las variables.
    
    Returns:
        X_opt (DataFrame): Conjunto óptimo de variables después de la eliminación.
        regressor_OLS (RegressionResultsWrapper): Modelo ajustado final.
    """
    numVars = X.shape[1]
    temp = X.copy()
    for i in range(numVars):
        regressor_OLS = sm.OLS(y, temp).fit()
        max_p_value = regressor_OLS.pvalues.max()
        if max_p_value > sl:
            excluded_feature = regressor_OLS.pvalues.idxmax()
            print(f"Eliminando '{excluded_feature}' con p-valor {max_p_value:.4f}")
            temp = temp.drop(columns=[excluded_feature])
        else:
            break
    print("\nResumen del modelo final:")
    print(regressor_OLS.summary())
    return temp, regressor_OLS

# Aplicar backward elimination
X_opt, model_opt = backward_elimination(X, y, sl=0.05)



Eliminando 'gdiagalt_S72.002A' con p-valor 0.9934
Eliminando 'ds_anemia' con p-valor 0.9888
Eliminando 'gdiagalt_S72.142A' con p-valor 0.9661
Eliminando 'ds_vitamina_d' con p-valor 0.9662
Eliminando 'ds_insuficiencia_cardiaca' con p-valor 0.9366
Eliminando 'ds_otras_alergias' con p-valor 0.9137
Eliminando 'ds_ITU' con p-valor 0.9073
Eliminando 'itipsexo' con p-valor 0.7891
Eliminando 'ds_centro_afueras' con p-valor 0.7563
Eliminando 'gdiagalt_S72.011A' con p-valor 0.7257
Eliminando 'gdiagalt_S72.102A' con p-valor 0.6305
Eliminando 'gsitalta' con p-valor 0.6014
Eliminando 'ds_alergia_alimenticia' con p-valor 0.5062
Eliminando 'ds_deterioro_cognitivo' con p-valor 0.4742
Eliminando 'ds_edad' con p-valor 0.4652
Eliminando 'ds_HTA' con p-valor 0.3412
Eliminando 'ds_insuficiencia_renal' con p-valor 0.4048
Eliminando 'const' con p-valor 0.2608
Eliminando 'ds_alergia_medicamentosa' con p-valor 0.2653
Eliminando 'ds_comorbilidades' con p-valor 0.2452
Eliminando 'ds_diabetes' con p-valor 0.4930


In [17]:
print("Variables finales:", X_opt.columns.tolist())


Variables finales: ['iotrocen', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_ulcera_presion', 'gdiagalt_S72.012A']


In [18]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Cargar el dataset y eliminar columnas no relevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_post_oper', 'ds_vivo_alta'])

# Asegurarse de que la variable respuesta es numérica y eliminar filas problemáticas
df["ds_pre_oper"] = pd.to_numeric(df["ds_pre_oper"], errors='coerce')
df = df.dropna(subset=["ds_pre_oper"])

# Separar la variable respuesta y los predictores
X = df.drop(columns=["ds_pre_oper"])
y = df["ds_pre_oper"]

# Convertir las variables categóricas a dummies (codificación one-hot)
X = pd.get_dummies(X, drop_first=True)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciar el modelo de regresión lineal
lr = LinearRegression()

# Aplicar la selección secuencial de características (forward selection)
sfs = SFS(lr,
          k_features='best',     # Se seleccionan las mejores combinaciones (se puede especificar un número fijo o 'best')
          forward=True,          # Para realizar selección hacia adelante
          floating=False,        # Sin retrocesos (puedes activarlo con floating=True)
          scoring='r2',          # Utilizamos R² como métrica de desempeño
          cv=5,                  # Validación cruzada de 5 pliegues
          n_jobs=-1)             # Utiliza todos los núcleos disponibles

sfs = sfs.fit(X_train, y_train)

# Imprimir las variables seleccionadas y la métrica en entrenamiento
print("Variables seleccionadas:", sfs.k_feature_names_)
print("R² en entrenamiento:", sfs.k_score_)

# Evaluar el modelo final en el set de prueba usando solo las variables seleccionadas
selected_features = list(sfs.k_feature_names_)
lr.fit(X_train[selected_features], y_train)
y_pred = lr.predict(X_test[selected_features])
print("R² en prueba:", r2_score(y_test, y_pred))


Variables seleccionadas: ('ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'gdiagalt_S72.012A')
R² en entrenamiento: 0.01827091442168931
R² en prueba: 0.04982597191858873


In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

# Cargar el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')

# Eliminar columnas no relevantes (ajusta según tus necesidades)
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_post_oper', 'ds_vivo_alta'])

# Asegurarse de que la variable respuesta sea numérica
df["ds_pre_oper"] = pd.to_numeric(df["ds_pre_oper"], errors='coerce')
df = df.dropna(subset=["ds_pre_oper"])

# Separar variables predictoras (X) y respuesta (y)
X = df.drop(columns=["ds_pre_oper"])
y = df["ds_pre_oper"]

# Convertir variables categóricas a dummies
X = pd.get_dummies(X, drop_first=True)

# Opcional: Eliminar filas con NaN si aparecen y sincronizar y
X = X.dropna()
y = y.loc[X.index]

# Dividir en conjunto de entrenamiento y prueba (para evaluación final)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el modelo de regresión lineal
lr = LinearRegression()

# Inicializar la selección de variables hacia adelante
remaining_features = list(X_train.columns)
selected_features = []
best_score = -np.inf  # score inicial muy bajo

print("Iniciando Forward Feature Selection:")
while remaining_features:
    scores_with_candidates = []
    for feature in remaining_features:
        current_features = selected_features + [feature]
        # Se evalúa el modelo con validación cruzada (5 pliegues) usando R²
        score = np.mean(cross_val_score(lr, X_train[current_features], y_train, cv=5, scoring='r2'))
        scores_with_candidates.append((score, feature))
    # Ordenar de mayor a menor score
    scores_with_candidates.sort(reverse=True)
    best_new_score, best_candidate = scores_with_candidates[0]
    # Si la mejora es mayor que la obtenida hasta ahora, se añade la variable
    if best_new_score > best_score:
        remaining_features.remove(best_candidate)
        selected_features.append(best_candidate)
        best_score = best_new_score
        print(f"  Se agregó: {best_candidate} con score = {best_new_score:.4f}")
    else:
        break

print("\nVariables seleccionadas:", selected_features)
print("R² (CV) del modelo seleccionado:", best_score)

# Evaluación final en el conjunto de prueba
lr.fit(X_train[selected_features], y_train)
y_pred = lr.predict(X_test[selected_features])
print("\nR² en conjunto de prueba:", r2_score(y_test, y_pred))


Iniciando Forward Feature Selection:
  Se agregó: ds_dia_semana_llegada con score = 0.0045
  Se agregó: ds_insuficiencia_respiratoria con score = 0.0120
  Se agregó: gdiagalt_S72.012A con score = 0.0158
  Se agregó: ds_insuficiencia_cardiaca con score = 0.0183

Variables seleccionadas: ['ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria', 'gdiagalt_S72.012A', 'ds_insuficiencia_cardiaca']
R² (CV) del modelo seleccionado: 0.01827091442168931

R² en conjunto de prueba: 0.04982597191858873


# RESUMEN


### DATOS_PREPROCESADOS_2_2022-2024


1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Funciones de selección usando statsmodels

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo R², RMSE y MAE en entrenamiento y en prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target: eliminar filas con valores nulos en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en el conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluación para el modelo forward
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en el conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluación para el modelo backward
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")


Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'iotrocen', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'ds_comorbilidades', 'ds_ulcera_presion', 'gdiagalt_S72.002A', 'gdiagalt_S72.011A', 'gdiagalt_S72.012A', 'gdiagalt_S72.101A', 'gdiagalt_S72.102A', 'gdiagalt_S72.141A', 'gdiagalt_S72.142A']

--- Forward Selection para ds_pre_oper en el conjunto de entrenamiento ---
Forward: Se agregó ds_dia_semana_llegada con AIC = 1564.04
Forward: Se agregó ds_insuficiencia_respiratoria con AIC = 1562.29
Forward: Se agregó ds_mes_llegada con AIC = 1561.03

Resumen del modelo final (Forward Selection):
                         

3. Selección Secuencial de Características con mlxtend


In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")


Procesando variable objetivo: ds_pre_oper


Variables seleccionadas: ['ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'gdiagalt_S72.012A']
R² (CV) en entrenamiento: 0.0183
R² en conjunto de prueba: 0.0498
Procesando variable objetivo: ds_post_oper
Variables seleccionadas: ['ds_centro_afueras', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'gdiagalt_S72.002A', 'gdiagalt_S72.101A']
R² (CV) en entrenamiento: 0.0906
R² en conjunto de prueba: -0.1488
Procesando variable objetivo: gsitalta
Variables seleccionadas: ['iotrocen', 'ds_alergia_alimenticia', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_ulcera_presion', 'gdiagalt_S72.002A', 'gdiagalt_S72.012A', 'gdiagalt_S72.102A']
R² (CV) en entrenamiento: 0.0118
R² en conjunto de prueba: -0.0823

Resumen de resultados:

Target: ds_pre_oper
Variables seleccionadas: ['ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_c

### DATOS_PREPROCESADOS_2


1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)



In [48]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Funciones de selección usando statsmodels

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo R², RMSE y MAE en entrenamiento y en prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target: eliminar filas con valores nulos en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en el conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluación para el modelo forward
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en el conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluación para el modelo backward
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")


Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'iotrocen', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'ds_comorbilidades', 'ds_ulcera_presion', 'gdiagalt_S72.002A', 'gdiagalt_S72.011A', 'gdiagalt_S72.012A', 'gdiagalt_S72.101A', 'gdiagalt_S72.102A', 'gdiagalt_S72.141A', 'gdiagalt_S72.142A']

--- Forward Selection para ds_pre_oper en el conjunto de entrenamiento ---
Forward: Se agregó ds_dia_semana_llegada con AIC = 3625.22
Forward: Se agregó ds_alergia_alimenticia con AIC = 3613.09
Forward: Se agregó ds_ulcera_presion con AIC = 3603.53
Forward: Se agregó ds_insuficiencia_respiratoria con AIC = 3598.41
Forward: 

3. Selección Secuencial de Características con mlxtend


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")


Procesando variable objetivo: ds_pre_oper
Variables seleccionadas: ['ds_turno', 'ds_edad', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_alergia_medicamentosa', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_diabetes', 'ds_comorbilidades', 'ds_ulcera_presion', 'gdiagalt_S72.002A', 'gdiagalt_S72.101A', 'gdiagalt_S72.141A']
R² (CV) en entrenamiento: 0.0426
R² en conjunto de prueba: -0.1286
Procesando variable objetivo: ds_post_oper
Variables seleccionadas: ['iotrocen', 'ds_otras_alergias', 'ds_ITU', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_ulcera_presion', 'gdiagalt_S72.011A', 'gdiagalt_S72.012A', 'gdiagalt_S72.102A', 'gdiagalt_S72.141A', 'gdiagalt_S72.142A']
R² (CV) en entrenamiento: 0.1227
R² en conjunto de prueba: 0.0404
Procesando variable objetivo: gsitalta
Variables seleccionadas: ['ds_edad', 'ds_dia_semana_llegada', 'ds_anemia', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_diabetes', 'gdiagalt_S72.002A']
R² (CV) en entrenamiento: -0.0202

### DATOS_PREPROCESADOS_2_2022-2024_MEDICAMENTOS


1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)



In [37]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo el R² en entrenamiento y prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluar el modelo forward en entrenamiento y prueba
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluar el modelo backward en entrenamiento y prueba
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024_MEDICAMENTOS.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")


Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'iotrocen', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'gfarmaco_0000000011', 'gfarmaco_15020', 'gfarmaco_15021', 'gfarmaco_504857', 'gfarmaco_600080.9', 'gfarmaco_600098', 'gfarmaco_600247', 'gfarmaco_600299.5', 'gfarmaco_600429', 'gfarmaco_600973', 'gfarmaco_602561', 'gfarmaco_602684.7', 'gfarmaco_603498', 'gfarmaco_603512.2', 'gfarmaco_603554.2', 'gfarmaco_604306', 'gfarmaco_607135', 'gfarmaco_607426', 'gfarmaco_6084144', 'gfarmaco_609222', 'gfarmaco_612283', 'gfarmaco_615096', 'gfarmaco_615799', 'gfarmaco_6162682', 'gfarmaco_616789', 'gfarmaco_6170199', 'gfarm

3. Selección Secuencial de Características con mlxtend


In [36]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_2022-2024_MEDICAMENTOS.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")


Procesando variable objetivo: ds_pre_oper


Variables seleccionadas: ['iotrocen', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'gfarmaco_15020', 'gfarmaco_504857', 'gfarmaco_600299.5', 'gfarmaco_603512.2', 'gfarmaco_603554.2', 'gfarmaco_609222', 'gfarmaco_616789', 'gfarmaco_618694', 'gfarmaco_623025', 'gfarmaco_6379189', 'gfarmaco_642504', 'gfarmaco_643536', 'gfarmaco_646331', 'gfarmaco_650097', 'gfarmaco_6514795', 'gfarmaco_662593', 'gfarmaco_669747', 'gfarmaco_686394', 'gfarmaco_691220', 'gfarmaco_694646', 'gfarmaco_741116', 'gfarmaco_804328', 'gfarmaco_944702', 'gfarmaco_979302', 'gfarmaco_P00002', 'gfarmaco_P667865']
R² (CV) en entrenamiento: 0.1554
R² en conjunto de prueba: 0.1671
Procesando variable objetivo: ds_post_oper
Variables seleccionadas: ['ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'gfarmaco_15020', 'gfarmaco_600080.9', 'gfarmaco_607426', 'gfarmaco_6170199', 'gfarmaco_622597', 'gfarmaco_624833', 'gfarmaco_6

### DATOS_PREPROCESADOS_3

1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)


In [43]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Funciones de selección usando statsmodels

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo R², RMSE y MAE en entrenamiento y en prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target: eliminar filas con valores nulos en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en el conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluación para el modelo forward
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en el conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluación para el modelo backward
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")



Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'movilidad', 'Barthel', 'braden', 'riesgo_caida', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'ds_comorbilidades', 'ds_ulcera_presion', 'gdiagalt_S72.012A', 'gdiagalt_S72.141A', 'gdiagalt_S72.142A']

--- Forward Selection para ds_pre_oper en el conjunto de entrenamiento ---
Forward: Se agregó ds_dia_semana_llegada con AIC = 928.38
Forward: Se agregó ds_alergia_alimenticia con AIC = 925.77
Forward: Se agregó ds_insuficiencia_respiratoria con AIC = 922.90
Forward: Se agregó ds_HTA con AIC = 921.23
Forward: Se agregó ds_diabetes con AIC = 919.40
Forward: Se agregó ds_

3. Selección Secuencial de Características con mlxtend


In [44]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")



Procesando variable objetivo: ds_pre_oper
Variables seleccionadas: ['ds_post_oper', 'ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria']
R² (CV) en entrenamiento: 0.0249
R² en conjunto de prueba: 0.0203
Procesando variable objetivo: ds_post_oper
Variables seleccionadas: ['ds_alergia_medicamentosa', 'ds_otras_alergias', 'ds_ITU']
R² (CV) en entrenamiento: 0.0193
R² en conjunto de prueba: 0.1636
Procesando variable objetivo: gsitalta
Variables seleccionadas: ['ds_izq_der', 'ds_dia_semana_llegada', 'ds_centro_afueras', 'ds_alergia_alimenticia', 'ds_vitamina_d', 'gdiagalt_S72.012A', 'gdiagalt_S72.142A']
R² (CV) en entrenamiento: 0.0432
R² en conjunto de prueba: -0.1272

Resumen de resultados:

Target: ds_pre_oper
Variables seleccionadas: ['ds_post_oper', 'ds_dia_semana_llegada', 'ds_insuficiencia_respiratoria']
R² (CV) en entrenamiento: 0.0249
R² en conjunto de prueba: 0.0203

Target: ds_post_oper
Variables seleccionadas: ['ds_alergia_medicamentosa', 'ds_otras_alergias', 'ds_ITU']
R² 

### DATOS_PREPROCESADOS_3_MEDICAMENTOS

1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)


In [45]:
### DATOS_PREPROCESADOS_3


import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Funciones de selección usando statsmodels

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo R², RMSE y MAE en entrenamiento y en prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target: eliminar filas con valores nulos en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en el conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluación para el modelo forward
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en el conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluación para el modelo backward
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3_MEDICAMENTOS.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")



Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'movilidad', 'Barthel', 'braden', 'riesgo_caida', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'gfarmaco_0000000011', 'gfarmaco_15020', 'gfarmaco_15021', 'gfarmaco_504857', 'gfarmaco_600080.9', 'gfarmaco_600098', 'gfarmaco_600247', 'gfarmaco_600299.5', 'gfarmaco_600429', 'gfarmaco_600973', 'gfarmaco_602561', 'gfarmaco_602684.7', 'gfarmaco_603498', 'gfarmaco_603512.2', 'gfarmaco_603554.2', 'gfarmaco_604306', 'gfarmaco_607135', 'gfarmaco_607426', 'gfarmaco_6084144', 'gfarmaco_609222', 'gfarmaco_612283', 'gfarmaco_615096', 'gfarmaco_615799', 'gfarmaco_6162682', 'gfarma

3. Selección Secuencial de Características con mlxtend

In [46]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PREPROCESADOS_3_MEDICAMENTOS.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")




Procesando variable objetivo: ds_pre_oper
Variables seleccionadas: ['ds_post_oper', 'ds_dia_semana_llegada', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'gfarmaco_15020', 'gfarmaco_600247', 'gfarmaco_600299.5', 'gfarmaco_600973', 'gfarmaco_603512.2', 'gfarmaco_607426', 'gfarmaco_615799', 'gfarmaco_6162682', 'gfarmaco_618694', 'gfarmaco_625277', 'gfarmaco_6254592', 'gfarmaco_6379189', 'gfarmaco_6409206', 'gfarmaco_6432969', 'gfarmaco_643536', 'gfarmaco_646331', 'gfarmaco_6472224', 'gfarmaco_648964', 'gfarmaco_649483', 'gfarmaco_651478', 'gfarmaco_6514795', 'gfarmaco_654047', 'gfarmaco_659755', 'gfarmaco_684653', 'gfarmaco_694646', 'gfarmaco_723308', 'gfarmaco_746214', 'gfarmaco_766949', 'gfarmaco_795831', 'gfarmaco_818997', 'gfarmaco_842500', 'gfarmaco_897314', 'gfarmaco_960351', 'gfarmaco_979302', 'gfarmaco_INSULINA', 'gfarmaco_P00002', 'gfarmaco_P659131.4', 'gfarmaco_P660686.5', 'gfarmaco_P667865']
R² (CV) en entrenamiento: 0.1918
R² en conjunto de prueba: 0.0125
Pr

### DATOS_PREPROCESADOS_3_SIN_COVID (Solo hay datos 2019)

1. Stepwise Regression (Forward y Backward Elimination con statsmodels) y 2. Backward Elimination Detallada (Ejemplo de Grover Jatin)


In [ ]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Funciones de selección usando statsmodels

def forward_selection(X, y, sl=0.05):
    """
    Realiza forward selection basándose en la reducción del AIC.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    remaining_features = list(X.columns)
    remaining_features.remove("const")  # ya se agregó la constante
    selected_features = ["const"]
    best_score = float('inf')
    while remaining_features:
        scores = []
        for feature in remaining_features:
            features_to_test = selected_features + [feature]
            model = sm.OLS(y, X[features_to_test]).fit()
            score = model.aic  # se puede usar AIC o BIC
            scores.append((score, feature))
        scores.sort()
        best_new_score, best_feature = scores[0]
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            print(f"Forward: Se agregó {best_feature} con AIC = {best_new_score:.2f}")
        else:
            break
    model_final = sm.OLS(y, X[selected_features]).fit()
    print("\nResumen del modelo final (Forward Selection):")
    print(model_final.summary())
    return selected_features, model_final

def backward_elimination(X, y, sl=0.05):
    """
    Realiza backward elimination eliminando iterativamente la variable con p-valor mayor a sl.
    Devuelve la lista de variables seleccionadas (incluyendo la constante) y el modelo final.
    """
    features = list(X.columns)
    while True:
        model = sm.OLS(y, X[features]).fit()
        pvals = model.pvalues.drop("const", errors='ignore')
        max_p = pvals.max()
        if max_p > sl:
            worst_feature = pvals.idxmax()
            features.remove(worst_feature)
            print(f"Backward: Se eliminó {worst_feature} (p = {max_p:.4f})")
        else:
            break
    model_final = sm.OLS(y, X[features]).fit()
    print("\nResumen del modelo final (Backward Elimination):")
    print(model_final.summary())
    return features, model_final

def run_statsmodels_selection(target, df, test_size=0.2, random_state=42, sl=0.05):
    """
    Ejecuta la preparación de datos y ambas técnicas (forward y backward) para un target dado.
    
    Parámetros:
      - target: nombre de la variable objetivo (string).
      - df: DataFrame original (con las columnas irrelevantes ya eliminadas).
    
    Imprime y devuelve un diccionario con los resultados, incluyendo R², RMSE y MAE en entrenamiento y en prueba.
    """
    print("="*80)
    print(f"Procesando target: {target}")
    
    # Crear un DataFrame local para el target: eliminar filas con valores nulos en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Forzar que todas las columnas de X sean numéricas
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Eliminar filas con NaN en X y sincronizar y
    X = X.dropna()
    y = y.loc[X.index]
    
    # Forzar conversión a float
    X = X.astype(float)
    y = y.astype(float)
    
    # Agregar la constante
    X = sm.add_constant(X)
    
    print("\nVariables a usar para", target, ":", X.columns.tolist())
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("\n--- Forward Selection para", target, "en el conjunto de entrenamiento ---")
    selected_forward, model_forward = forward_selection(X_train, y_train, sl=sl)
    recommended_forward = [var for var in selected_forward if var != "const"]
    
    # Evaluación para el modelo forward
    y_train_pred_forward = model_forward.predict(X_train[selected_forward])
    r2_train_forward = r2_score(y_train, y_train_pred_forward)
    y_test_pred_forward = model_forward.predict(X_test[selected_forward])
    r2_test_forward = r2_score(y_test, y_test_pred_forward)
    
    print("\n--- Backward Elimination para", target, "en el conjunto de entrenamiento ---")
    selected_backward, model_backward = backward_elimination(X_train, y_train, sl=sl)
    recommended_backward = [var for var in selected_backward if var != "const"]
    
    # Evaluación para el modelo backward
    y_train_pred_backward = model_backward.predict(X_train[selected_backward])
    r2_train_backward = r2_score(y_train, y_train_pred_backward)
    y_test_pred_backward = model_backward.predict(X_test[selected_backward])
    r2_test_backward = r2_score(y_test, y_test_pred_backward)
    
    results = {
        "target": target,
        "recommended_forward": recommended_forward,
        "forward_model_summary": model_forward.summary().as_text(),
        "r2_train_forward": r2_train_forward,
        "r2_test_forward": r2_test_forward,
        "recommended_backward": recommended_backward,
        "backward_model_summary": model_backward.summary().as_text(),
        "r2_train_backward": r2_train_backward,
        "r2_test_backward": r2_test_backward
    }
    
    print("\nMétricas Forward:")
    print(f"R² en entrenamiento: {r2_train_forward:.4f}, R² en conjunto de prueba: {r2_test_forward:.4f}")
    print("\nMétricas Backward:")
    print(f"R² en entrenamiento: {r2_train_backward:.4f}, R² en conjunto de prueba: {r2_test_backward:.4f}")
    
    return results

# Cargar el dataset original y eliminar columnas irrelevantes
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/preprocesado/DATOS_PRPROCESADOS_3_SIN_COVID.csv')
df = df.drop(columns=['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta'])
# NOTA: Para este ejemplo conservamos ds_pre_oper, ds_post_oper y gsitalta como variables objetivo.
# Asegurarse de que las variables objetivo sean numéricas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar la selección para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
all_results = {}

for target in targets:
    all_results[target] = run_statsmodels_selection(target, df, sl=0.05)

# Resumen de resultados (incluyendo R² en entrenamiento y prueba para forward y backward)
print("\nResumen de resultados:")
for target, res in all_results.items():
    print(f"\nTarget: {target}")
    print("Variables recomendadas (Forward):", res["recommended_forward"])
    print(f"R² (Forward) - Entrenamiento: {res['r2_train_forward']:.4f}, Prueba: {res['r2_test_forward']:.4f}")
    print("Variables recomendadas (Backward):", res["recommended_backward"])
    print(f"R² (Backward) - Entrenamiento: {res['r2_train_backward']:.4f}, Prueba: {res['r2_test_backward']:.4f}")



Procesando target: ds_pre_oper

Variables a usar para ds_pre_oper : ['const', 'itipsexo', 'gsitalta', 'ds_izq_der', 'ds_turno', 'ds_edad', 'ds_post_oper', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'movilidad', 'Barthel', 'braden', 'riesgo_caida', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes', 'gfarmaco_0000000011', 'gfarmaco_15020', 'gfarmaco_15021', 'gfarmaco_504857', 'gfarmaco_600080.9', 'gfarmaco_600098', 'gfarmaco_600247', 'gfarmaco_600299.5', 'gfarmaco_600429', 'gfarmaco_600973', 'gfarmaco_602561', 'gfarmaco_602684.7', 'gfarmaco_603498', 'gfarmaco_603512.2', 'gfarmaco_603554.2', 'gfarmaco_604306', 'gfarmaco_607135', 'gfarmaco_607426', 'gfarmaco_6084144', 'gfarmaco_609222', 'gfarmaco_612283', 'gfarmaco_615096', 'gfarmaco_615799', 'gfarmaco_6162682', 'gfarma

/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1527: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params * scale


                            OLS Regression Results                            
Dep. Variable:            ds_pre_oper   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 18 Mar 2025   Prob (F-statistic):                nan
Time:                        08:18:31   Log-Likelihood:                 2504.4
No. Observations:                  92   AIC:                            -4825.
Df Residuals:                       0   BIC:                            -4593.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/s

Forward: Se agregó ds_insuficiencia_renal con AIC = 314.34
Forward: Se agregó gdiagalt_S72.142A con AIC = 304.68
Forward: Se agregó gfarmaco_6254592 con AIC = 298.57
Forward: Se agregó gfarmaco_979328 con AIC = 293.14
Forward: Se agregó gfarmaco_622597 con AIC = 283.80
Forward: Se agregó gfarmaco_652001.7 con AIC = 276.50
Forward: Se agregó gfarmaco_953794 con AIC = 271.53
Forward: Se agregó gfarmaco_602561 con AIC = 268.95
Forward: Se agregó gfarmaco_603554.2 con AIC = 258.19
Forward: Se agregó gfarmaco_P619239 con AIC = 254.18
Forward: Se agregó gfarmaco_835835 con AIC = 248.46
Forward: Se agregó gfarmaco_960344 con AIC = 242.61
Forward: Se agregó gfarmaco_631192 con AIC = 239.45
Forward: Se agregó gfarmaco_616789 con AIC = 235.29
Forward: Se agregó gfarmaco_649483 con AIC = 232.85
Forward: Se agregó gfarmaco_0000000011 con AIC = 229.72
Forward: Se agregó gfarmaco_651478 con AIC = 227.23
Forward: Se agregó ds_centro_afueras con AIC = 223.77
Forward: Se agregó ds_insuficiencia_cardiac

/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/s

3. Selección Secuencial de Características con mlxtend


In [ ]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def run_sfs_for_target(target, df, test_size=0.2, random_state=42):
    """
    Preprocesa el dataset para el target dado, aplica Sequential Feature Selector (SFS)
    y evalúa el modelo de regresión lineal.

    Parámetros:
      - target: Nombre de la variable objetivo.
      - df: DataFrame original.
      - test_size: Proporción del conjunto de prueba.
      - random_state: Semilla para la división.

    Imprime:
      - Variables seleccionadas.
      - R² (CV) en entrenamiento (promedio de validación cruzada).
      - R² en el conjunto de prueba.
    Devuelve un diccionario con los resultados.
    """
    print("="*80)
    print(f"Procesando variable objetivo: {target}")
    
    # Crear una copia del DataFrame y eliminar filas con NaN en el target
    df_target = df.copy().dropna(subset=[target])
    
    # Convertir el target a numérico (en caso de ser necesario)
    df_target[target] = pd.to_numeric(df_target[target], errors='coerce')
    df_target = df_target.dropna(subset=[target])
    
    # Separar X e y: los predictores son todas las columnas excepto el target
    X = pd.get_dummies(df_target.drop(columns=[target]), drop_first=True)
    y = df_target[target]
    
    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Instanciar el modelo
    lr = LinearRegression()
    
    # Aplicar SFS (forward selection) usando validación cruzada (5-fold) con R²
    sfs = SFS(lr,
              k_features='best',
              forward=True,
              floating=False,
              scoring='r2',
              cv=5,
              n_jobs=-1)
    sfs = sfs.fit(X_train, y_train)
    
    selected_features = list(sfs.k_feature_names_)
    cv_score = sfs.k_score_
    
    # Entrenar el modelo final con las características seleccionadas
    lr.fit(X_train[selected_features], y_train)
    y_pred = lr.predict(X_test[selected_features])
    
    test_r2 = r2_score(y_test, y_pred)
    
    print("Variables seleccionadas:", selected_features)
    print(f"R² (CV) en entrenamiento: {cv_score:.4f}")
    print(f"R² en conjunto de prueba: {test_r2:.4f}")
    
    return {
        "target": target,
        "selected_features": selected_features,
        "cv_r2": cv_score,
        "test_r2": test_r2
    }

# Cargar y preprocesar el dataset original
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_3/DATOS_PRPROCESADOS_3_SIN_COVID.csv')
# Eliminar columnas irrelevantes
cols_to_drop = ['gidenpac', 'fllegada', 'ds_estancia', 'ds_vivo_alta']
df = df.drop(columns=cols_to_drop)

# Para este ejemplo, conservamos tres variables objetivo: ds_pre_oper, ds_post_oper y gsitalta.
# Convertirlas a numérico y eliminar filas con problemas:
for target in ["ds_pre_oper", "ds_post_oper", "gsitalta"]:
    df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=["ds_pre_oper", "ds_post_oper", "gsitalta"])

# Ejecutar el proceso para cada variable objetivo
targets = ["ds_pre_oper", "ds_post_oper", "gsitalta"]
results = {}

for target in targets:
    results[target] = run_sfs_for_target(target, df)

print("\nResumen de resultados:")
for target, res in results.items():
    print(f"\nTarget: {target}")
    print(f"Variables seleccionadas: {res['selected_features']}")
    print(f"R² (CV) en entrenamiento: {res['cv_r2']:.4f}")
    print(f"R² en conjunto de prueba: {res['test_r2']:.4f}")



Procesando variable objetivo: ds_pre_oper
Variables seleccionadas: ['itipsexo', 'ds_vitamina_d', 'ds_insuficiencia_cardiaca', 'ds_HTA', 'ds_diabetes', 'gfarmaco_0000000011', 'gfarmaco_504857', 'gfarmaco_600247', 'gfarmaco_600299.5', 'gfarmaco_604306', 'gfarmaco_607426', 'gfarmaco_6084144', 'gfarmaco_6162682', 'gfarmaco_618694', 'gfarmaco_621904', 'gfarmaco_642736', 'gfarmaco_6432969', 'gfarmaco_6472224', 'gfarmaco_649335', 'gfarmaco_650097', 'gfarmaco_702951', 'gfarmaco_723308', 'gfarmaco_746214', 'gfarmaco_822528', 'gfarmaco_835835', 'gfarmaco_842500', 'gfarmaco_878728', 'gfarmaco_8888888889', 'gfarmaco_897314', 'gfarmaco_953794', 'gfarmaco_979328', 'gfarmaco_9999999999', 'gfarmaco_INSULINA', 'gdiagalt_S72.141A']
R² (CV) en entrenamiento: 0.2008
R² en conjunto de prueba: -0.4725
Procesando variable objetivo: ds_post_oper
